<a href="https://colab.research.google.com/github/wonseok0802/my-first-repo/blob/main/%5BHands_on_Practice_06%5D_IR_Sparse_vs_Dense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## << 문제 정의 >>

- **Retrieval(검색)** 은 주어진 쿼리에 대해 관련된 문서를 찾아오는 태스크입니다.

- 전통적인 **TF-IDF** 방식과 딥러닝 기반의 **SentenceBERT** 방식의 검색 성능을 비교합니다.

- 두 방식의 차이점을 이해하고, 각각의 장단점을 파악합니다.

### TF-IDF vs SentenceBERT 비교

| 구분 | TF-IDF | SentenceBERT |
|------|--------|---------------|
| 원리 | 단어 빈도 기반 (Sparse) | 의미 기반 (Dense) |
| 벡터 차원 | 어휘 크기 (수만 차원) | 고정 크기 (384~768) |
| 동의어 처리 | ❌ 불가능 | ✅ 가능 |
| 전처리 의존도 | 높음 (형태소 분석 필수) | 낮음 |
| 계산 속도 | 빠름 | 상대적으로 느림 |
| 메모리 사용 | Sparse로 효율적 | Dense로 더 많이 사용 |

### << 진행 순서 >>

1. 환경 설정 및 데이터 준비 (Ko-StrategyQA)

2. 한국어 전처리 (Kiwi 형태소 분석)

3. TF-IDF 기반 Retrieval 구현 (전처리 유무 비교)

4. SentenceBERT 기반 Retrieval 구현 (한국어 전용 모델)

5. 동일 쿼리에 대한 검색 결과 비교

6. 정량적 성능 평가 (Recall@K, MRR)

7. 임베딩 공간 시각화 비교

---
## Part 1. 환경 설정

In [ ]:
!pip install kiwipiepy sentence-transformers datasets

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
import json
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (시각화용)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

---
## Part 2. 데이터 준비

### Ko-StrategyQA 데이터셋

- **한국어 Retrieval 벤치마크** 데이터셋 (BeIR 포맷)
- **corpus**: 9,250개 문서 (Wikipedia 기반)
- **queries**: 592개 쿼리
- **qrels**: 쿼리-문서 관련성 판단 (1개 쿼리당 여러 개의 relevant docs)
- Multi-hop QA 데이터를 Retrieval용으로 변환

### Q1. Ko-StrategyQA 데이터셋을 로드하고 corpus, queries, qrels 데이터를 확인하는 코드를 작성하세요.

- `corpus.jsonl`: 검색 대상 문서들
- `queries.jsonl`: 검색 쿼리들
- `qrels/`: 쿼리-문서 관련성 레이블

[HINT] HuggingFace datasets 라이브러리의 `load_dataset` 함수를 사용합니다. 데이터셋 이름은 `'mteb/Ko-StrategyQA'`입니다.

In [ ]:
# A1.


In [ ]:
# 데이터 구조 확인
print("=" * 50)
print("Corpus 예시:")
print(corpus_data[0])
print("\n" + "=" * 50)
print("Query 예시:")
print(queries_data[0])
print("\n" + "=" * 50)
print("Qrels 예시:")
print(qrels_data[0])

### Q2. 검색에 사용할 데이터 구조를 만드세요. 아래 설명을 참고하세요.

1. `corpus`: doc_id → text 매핑 (딕셔너리)
2. `corpus_list`: 문서 텍스트 리스트 (임베딩용)
3. `doc_id_list`: doc_id 순서 유지 (리스트)
4. `queries`: query_id → query text 매핑 (딕셔너리)
5. `qrels`: query_id → relevant doc_ids 매핑 (딕셔너리, set으로 저장)

[HINT] corpus 데이터의 각 item에는 `_id`, `title`, `text` 필드가 있습니다. title과 text를 합쳐서 문서를 구성하세요.

In [ ]:
# A2.


---
## Part 3. 한국어 전처리 (Kiwi 형태소 분석)

TF-IDF는 단어(토큰) 기반으로 동작하기 때문에, 한국어의 경우 **형태소 분석**이 매우 중요합니다.

### Kiwi 형태소 분석기

- **높은 정확도**: 웹 텍스트 87%, 문어 텍스트 94%
- **빠른 속도**: C++ 기반, 멀티스레딩 지원
- **간편한 설치**: `pip install kiwipiepy`

### Q3. Kiwi 형태소 분석기를 사용하여 문장을 토큰화하는 함수를 구현하세요. 아래 설명을 참고하세요.

1. `kiwipiepy.Kiwi`를 사용합니다.
2. 명사(NN*), 동사(VV), 형용사(VA)만 추출합니다.
3. 불용어(stopwords)를 제거합니다.
4. 길이가 1 이하인 토큰은 제거합니다.

[HINT] `kiwi.tokenize(text)`를 사용하면 토큰 리스트를 얻을 수 있고, 각 토큰의 `.form`(형태), `.tag`(품사)를 확인할 수 있습니다.

In [ ]:
# A3.


### Q4. 전체 코퍼스에 형태소 분석을 적용하는 코드를 작성하세요.

[HINT] `tqdm`을 사용하면 진행 상황을 확인할 수 있습니다. 리스트 컴프리헨션을 사용하세요.

In [ ]:
# A4.


---
## Part 4. TF-IDF 기반 Retrieval 구현

### Q5. TF-IDF Vectorizer를 사용하여 코퍼스를 벡터화하세요. 아래 설명을 참고하세요.

1. 전처리 없는 TF-IDF (`tfidf_vectorizer_raw`, `tfidf_matrix_raw`)
2. Kiwi 형태소 분석 후 TF-IDF (`tfidf_vectorizer`, `tfidf_matrix`)

[HINT] `sklearn.feature_extraction.text.TfidfVectorizer`를 사용합니다.

In [ ]:
# A5.


### Q6. TF-IDF 기반 검색 함수를 구현하세요. 아래 설명을 참고하세요.

1. `query`: 검색할 쿼리 문자열
2. `top_k`: 반환할 상위 문서 개수 (기본값 10)
3. `use_preprocessing`: 전처리 적용 여부 (기본값 True)
4. 반환값: `[(doc_id, score), ...]` 형태의 리스트

[HINT] `sklearn.metrics.pairwise.cosine_similarity`를 사용하여 쿼리와 문서 간의 유사도를 계산합니다.

In [ ]:
# A6.


---
## Part 5. SentenceBERT 기반 Retrieval 구현

### Q7. 한국어 전용 SentenceBERT 모델을 로드하고 코퍼스를 임베딩하세요. 아래 설명을 참고하세요.

1. `sentence_transformers.SentenceTransformer`를 사용합니다.
2. 한국어 전용 모델을 사용합니다.
3. 전체 코퍼스를 임베딩합니다.

[HINT] 한국어 전용 모델로 `'snunlp/KR-SBERT-V40K-klueNLI-augSTS'`를 사용할 수 있습니다. 임베딩에는 약 3-5분이 소요됩니다.

In [ ]:
# A7.


### Q8. SentenceBERT 기반 검색 함수를 구현하세요. 아래 설명을 참고하세요.

1. 쿼리를 임베딩합니다.
2. 코사인 유사도를 계산합니다.
3. 상위 K개의 문서를 반환합니다.

[HINT] `sentence_transformers.util.pytorch_cos_sim`을 사용하면 코사인 유사도를 쉽게 계산할 수 있습니다. `torch.topk`로 상위 K개를 추출합니다.

In [ ]:
# A8.


---
## Part 6. 검색 결과 비교

### Q9. 동일한 쿼리에 대해 세 가지 방식(TF-IDF 전처리X, TF-IDF Kiwi, SentenceBERT)의 검색 결과를 비교하는 함수를 구현하세요.

[HINT] 각 방식의 검색 결과에서 relevant docs에 포함된 문서는 ✓ 표시를 해주면 비교하기 좋습니다.

In [ ]:
# A9.


---
## Part 7. 정량적 성능 평가

### 평가 지표

- **Recall@K**: 전체 relevant docs 중 상위 K개 안에 포함된 비율
- **MRR**: 첫 번째 relevant doc이 등장하는 순위의 역수 평균

### Q10. Recall@K와 MRR을 계산하는 함수를 구현하세요.

**Recall@K 공식:**
$$Recall@K = \frac{|\{relevant\_docs\} \cap \{top\_K\_retrieved\}|}{|\{relevant\_docs\}|}$$

**MRR 공식:**
$$MRR = \frac{1}{|Q|} \sum_{i=1}^{|Q|} \frac{1}{rank_i}$$

[HINT] Recall@K는 상위 K개 중 relevant docs가 몇 개인지 비율로 계산합니다. MRR은 첫 번째 relevant doc의 순위 역수를 평균냅니다.

In [ ]:
# A10.


### Q11. 세 가지 방식의 성능을 평가하고 비교하세요. K값은 1, 3, 5, 10으로 설정하세요.

[HINT] 각 방식별로 Recall@K와 MRR을 계산하여 DataFrame으로 정리하면 보기 좋습니다.

In [ ]:
# A11.


### Q12. 성능 비교 결과를 시각화하세요. Recall@K는 막대 그래프로, MRR은 별도의 막대 그래프로 표현하세요.

[HINT] `matplotlib.pyplot.subplots`를 사용하여 1x2 레이아웃으로 그래프를 그릴 수 있습니다.

In [ ]:
# A12.


---
## Part 8. 임베딩 공간 시각화

### Q13. t-SNE를 사용하여 TF-IDF와 SentenceBERT의 임베딩 공간을 2D로 시각화하세요.

[HINT] 전체 데이터를 사용하면 시간이 오래 걸리므로, 500개 정도 샘플링하여 사용합니다. `sklearn.manifold.TSNE`를 사용하세요.

In [ ]:
# A13.


---
## (optional)>> [Advanced] 하이브리드 검색

### Q14. TF-IDF와 SentenceBERT를 결합한 하이브리드 검색 함수를 구현하세요. 아래 설명을 참고하세요.

**하이브리드 점수 공식:**
$$score = \alpha \times tfidf\_score + (1-\alpha) \times sbert\_score$$

1. TF-IDF 점수와 SentenceBERT 점수를 각각 계산합니다.
2. Min-Max 정규화를 적용합니다.
3. 가중 평균으로 최종 점수를 계산합니다.

[HINT] `alpha`는 TF-IDF의 가중치로, 0이면 SentenceBERT만, 1이면 TF-IDF만 사용합니다.

In [ ]:
# A14.


### Q15. 최적의 alpha 값을 찾는 실험을 수행하세요. alpha를 0.0부터 1.0까지 0.1 간격으로 변화시키며 Recall@5를 측정하세요.

[HINT] 각 alpha 값에 대해 하이브리드 검색의 Recall@5를 계산하고, 최적의 alpha를 찾습니다.

In [ ]:
# A15.
